# scikit-learn (sklearn) で線形回帰（重回帰）を動かす
**scikit-learn (sklearn)はPythonの機械学習ライブラリ**です．公式：https://scikit-learn.org/stable/

sklearnを用いると，驚くべきことに，**ライブラリのインポート・データの読み込みを除けば10行未満で学習・予測**まで行うことができます．
Pythonなので遅いのではないか（自分でCで書いた方が速いのではないか）と思うかもしれませんが，sklearnではNumPyやSciPyを上手く用いて書かれたコードや，Cで書かれた大変質の良いコードが動いているため，sklearnはかなり高速に動作します．
ほとんどの場合，雑に書いたCのプログラムよりは高速に動作します．
またPythonに慣れていなくとも，「慣れていないPythonでscikit-learnを使えるようになる時間」と「慣れているCで実装する時間」だと，おそらく圧倒的に前者の方が短いでしょう．

勿論，勉強のためにsklearnを使わずに実装しても構いませんが，本演習ではデータ分析全体の流れを体験してもらいたいため，サンプルコードではsklearnを用います．

## 今回学ぶこと
- pandasによるcsvの読み込みと簡単な操作（列・行へのアクセス，`numpy.ndarray`への変換）
- sklearnの基本的な使い方
- 提出までの流れ：
  1. 特徴ベクトルの作成
  2. モデル・アルゴリズムの選定，学習
  4. 予測
  5. 提出

## 予測モデル構築の流れ

本演習のように，機械学習を用いて予測モデルを構築し，未知の（テスト）データに対して予測を行う手順は，主に以下のようになります．

1. データを用意し，特徴ベクトルを作る
2. どのような手法（モデル）を使うかを決める
3. モデルを学習する方法を決め，学習する
4. 未知のデータに予測を行う（そして本演習では提出する）


## データ読み込み
演習で用いるデータはcsvファイル（comma-separated valuesファイル）で表現されています．
Pythonの標準の機能を使って読み込んでも良いのですが，Pythonには**pandasというデータ解析のための便利なライブラリ**があります．
pandasによって，CSVを含む様々なファイルを簡単に読み込み，さらに読み込んだデータに対して様々な処理を簡単に行うことができます．

なお，今回は文字列の情報を含んでいるためpandasを用いますが，数値情報だけであれば，Pythonにおける**数値・行列・線形代数計算ライブラリのNumPy**で読み込んで処理するほうが楽であることも多いです（NumPyについては，day1_2_how_to_numpy.ipynbやday1_2_how_to_numpy_full.ipynbを参照）．
また，scikit-learnの関数には，NumPyの配列を前提とするものやNumPyの配列を返すものも数多くあります．そのため，今回の演習ではpandasで最低限の処理をした後，NumPyの配列に変換し（=いろいろな演算がサポートされている便利な配列），変換したものをscikit-learnに渡すことにします．
つまり，以下のような手順を行います：
1. `pandas`でデータを読み込む
2. `pandas`の便利な関数でごにょごにょして，`numpy`の`ndarray`（`array`）に変換
3. （場合によっては）`numpy`の便利な関数でごにょごにょして，特徴ベクトルを作成
4. scikit-learnに渡して学習

まず，今回使うものを`import`（=C言語で言うところのincludeだと思ってください）をしましょう．

In [1]:
import numpy as np
import pandas as pd 
# 機械学習のライブラリ．今回は線形回帰LinearRegressionを動かす．
# LinearRegressionはsklearnのlinear_modelモジュールの中にあるので，次のようにインポートする
from sklearn.linear_model import LinearRegression 

上のセルを動かしましたか？その際に特にエラーが出なければ，インポートに成功しています．
これ以降皆さんはnumpy，pandas，そしてsklearn.linear_modelの`LinearRegression`が使えるようになりました！


では，次は訓練データとテストデータを読み込みます．
csvファイルは，pandasの`read_csv`という関数を使って読み込むことができます．
ファイルのパスを引数として渡すと，`pandas.DataFrame`という型で（オブジェクト指向言語なので，「`pandas.DataFrame`クラスのインスタンスを作成して」とも言える）読み込んだものを返してくれます．
詳細は[公式のドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)を参照してください．
では実際に動かしてみます．

このノートブックを動かしているディレクトリにdataディレクトリがあり，その中にtrain.csvとtest.csvが存在していると仮定しています．
そうでない場合は適宜書き換えてください．

Goole Colabを用いている場合，Google Driveにdataフォルダを置いて，次のセルのコメントアウトを外して動かしてください（自身のPython環境で動かしている場合は，次のセルは飛ばして良いです）．次のセルを動かした後，以下の手順を踏むことでGoogle Driveのデータを読み込むことができるようになります．
- 以下のセルを動かすと，URLが出てくるのでそこにアクセスしてください．
- グーグルアカウントのログインが求められるので，ログインしてください．
- authorization codeが表示されるので，それを貼り付けてください．
- 更にその下のセルの`read_csv`で指定されているパスを適宜書き換えてください．画面左側のバーで「フォルダ」のタブを選ぶと"drive"というフォルダが出てくると思いますが，そこがGoogle Driveを表しています．そこを見て適宜パスを書き換えてください．

In [2]:
#from google.colab import drive
#drive.mount('/content/drive') # google driveをマウント（＝Colabから使えるようにする）

In [3]:
# 動的型付け言語なので，変数の型の宣言は不要
d_train = pd.read_csv("data/train.csv") # 訓練データを読み込む
d_test = pd.read_csv("data/test.csv") # テストデータを読み込む
# Google Colabの場合
#d_train = pd.read_csv("drive/My Drive/data/train.csv") # 訓練データを読み込む．TFがGoogle Driveの一番上にdataディレクトリを置いた場合はこのようなパスになった
#d_test = pd.read_csv("drive/My Drive/data/test.csv") # テストデータを読み込む． TFがGoogle Driveの一番上にdataディレクトリを置いた場合はこのようなパスになった

エラーが何も出ていなければ読み込めています．しかし言われるがまま読み込んだだけでどうなっているのかよくわかりませんね．`print`してみましょう．

In [4]:
print("訓練データ")
print(d_train)
print("\nテストデータ")
print(d_test)

訓練データ
             Date         4046         4225       4770  Small Bags  \
0      2017-04-23      1121.34      5184.26       0.00      888.21   
1      2017-09-17   1582315.47   1095433.56   47072.46   863081.25   
2      2017-11-05       141.41       794.48       0.00     5320.50   
3      2016-01-24      5289.88       111.58       0.00     5096.27   
4      2018-01-28  14551799.50  12119884.61  575974.74  9749412.19   
...           ...          ...          ...        ...         ...   
12769  2015-07-12      1639.19        49.62       0.00     3592.55   
12770  2016-01-31      8791.51     58433.15       0.00     6581.99   
12771  2018-01-21       613.65      1852.11       0.00     4083.63   
12772  2016-07-31       553.48         5.70       0.00     4514.72   
12773  2015-03-15    156506.22    160239.43   17300.74    58756.28   

       Large Bags  XLarge Bags          type            region  AveragePrice  
0         2717.94         0.00       organic          LasVegas          1.

train.csvの中身が出てきました！すばらしいですね．
詳細はスライドを確認してほしいですが，この行列は，各行が一つのアボカドのデータに対応しています．
各列はアボカドの何らかの情報を表していて，各列がどのような情報なのかはヘッダーに書かれています．
たとえば，1列目は"Date"と書かれているので，これは日付の情報ですね．
訓練データの最後の列は"AveragePrice"と書かれており，これはまさしく目標値（出力・教師情報）です．
テストデータに"AveragePrice"の列はなく，この列の値を予測して，その予測の正確さで皆さんに競っていただくのがこのコンペティションです．


さて，予測モデルを動かす前に，ちょっと`d_train`で遊んでみます．各列や行を取り出してみましょう．

列を取り出すのは非常に簡単で，他のプログラミング言語で配列の要素にアクセスするのと同じようにブラケット（角括弧）を用います．具体的には，

d_train[列の名前]

とすることで取り出せます．また，複数の列を取り出すことは，

d_train[列の名前のリスト]

でできます（"取り出す"と言っていますが，元の`DataFrame`から消えるわけではありません）．以下のセルで実際にやってみましょう．

In [5]:
print("値段の列を取り出す")
print(d_train['AveragePrice'])

print("\nオーガニック否かと地域を取り出す")
print(d_train[["type", "region"]])

print("\n取り出した結果を別の変数に格納することもできる")
d_train_date = d_train["Date"]
print(d_train_date)


値段の列を取り出す
0        1.67
1        1.53
2        1.48
3        1.38
4        1.09
         ... 
12769    1.89
12770    1.45
12771    1.50
12772    1.41
12773    1.21
Name: AveragePrice, Length: 12774, dtype: float64

オーガニック否かと地域を取り出す
               type            region
0           organic          LasVegas
1      conventional              West
2           organic      Indianapolis
3           organic           Houston
4      conventional           TotalUS
...             ...               ...
12769       organic           Orlando
12770       organic        GreatLakes
12771       organic          Columbus
12772       organic  NewOrleansMobile
12773  conventional          Portland

[12774 rows x 2 columns]

取り出した結果を別の変数に格納することもできる
0        2017-04-23
1        2017-09-17
2        2017-11-05
3        2016-01-24
4        2018-01-28
            ...    
12769    2015-07-12
12770    2016-01-31
12771    2018-01-21
12772    2016-07-31
12773    2015-03-15
Name: Date, Length: 12774, dtype: object


とりだせました！すばらしいですね．

次に行を取り出してみましょう．行も列と同様にブラケット（角括弧）で取り出せます．d_train\[i:j\]とすることで，i行目からj-1行目の値を取り出すことができます．

In [6]:
print("0から4-1行目まで取り出す")
print(d_train[0:4])
print("\n10行目だけ取り出す")
print(d_train[10:11])

0から4-1行目まで取り出す
         Date        4046        4225      4770  Small Bags  Large Bags  \
0  2017-04-23     1121.34     5184.26      0.00      888.21     2717.94   
1  2017-09-17  1582315.47  1095433.56  47072.46   863081.25   591149.15   
2  2017-11-05      141.41      794.48      0.00     5320.50     1452.69   
3  2016-01-24     5289.88      111.58      0.00     5096.27        6.67   

   XLarge Bags          type        region  AveragePrice  
0         0.00       organic      LasVegas          1.67  
1      2174.92  conventional          West          1.53  
2         0.00       organic  Indianapolis          1.48  
3         0.00       organic       Houston          1.38  

10行目だけ取り出す
          Date      4046     4225   4770  Small Bags  Large Bags  XLarge Bags  \
10  2015-05-10  52698.39  52242.5  146.1    34415.41     2357.78          0.0   

            type   region  AveragePrice  
10  conventional  Roanoke          1.09  


できました．素晴らしいですね！他にも色々できますし，同じことを行う別の方法があったりもしますが，今回はとりあえずこの程度にしておきましょう．
詳細は例えばhttps://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html 等を見てみてください．

さて，次に特徴ベクトルを作っていきたいのですが，その前にいくつかの変数を準備します．

まず，訓練データとテストデータの数を表す変数`n_train`と`n_test`を準備します．今日は（最後の発展的なQuiz 3以外では）使いませんが，今後よく使うためここで取得の仕方を確認しておきます．
各行が一つのデータに対応しているので，**`d_train`や`d_test`の行の数**が取得できればよいのですが，これは`len`関数によって取得できます．

また，sklearnでモデルを学習させる際に必要となるので，訓練データの目標値"AveragePrice"を`d_train`から取り出して分離させておきます．
`pop`メソッドを使うことで，`DataFrame`（`d_train`や`d_test`）から特定の列を分離させることができます．
「分離」と言っている通り，ブラケットによるアクセスと違い，`pop`で取り出すと**元の`DataFrame`からその列は削除されます**．

In [7]:
print("訓練データとテストデータの数を取得")
n_train = len(d_train)
n_test = len(d_test)
print(f"訓練データ数：{n_train}，テストデータ数：{n_test}")
print("\n 出力情報を取り出す．AveragePriceの列が消えている")
# targetの値
y_train = d_train.pop('AveragePrice')
print(d_train)
print(y_train)
print(f"y_trainのクラスは{type(y_train)}")

訓練データとテストデータの数を取得
訓練データ数：12774，テストデータ数：5475

 出力情報を取り出す．AveragePriceの列が消えている
             Date         4046         4225       4770  Small Bags  \
0      2017-04-23      1121.34      5184.26       0.00      888.21   
1      2017-09-17   1582315.47   1095433.56   47072.46   863081.25   
2      2017-11-05       141.41       794.48       0.00     5320.50   
3      2016-01-24      5289.88       111.58       0.00     5096.27   
4      2018-01-28  14551799.50  12119884.61  575974.74  9749412.19   
...           ...          ...          ...        ...         ...   
12769  2015-07-12      1639.19        49.62       0.00     3592.55   
12770  2016-01-31      8791.51     58433.15       0.00     6581.99   
12771  2018-01-21       613.65      1852.11       0.00     4083.63   
12772  2016-07-31       553.48         5.70       0.00     4514.72   
12773  2015-03-15    156506.22    160239.43   17300.74    58756.28   

       Large Bags  XLarge Bags          type            region  
0         2717.94

訓練データ数は`12774`，テストデータ数は`5475`で，これはスライドやコンペサイトに書いてあるとおりです．
また，`d_train`から"AveragePrice"の列が消えていて，`y_train`が"AveragePrice"の情報を持っていますね．

今，`y_train`は怪しい（？）クラスのインスタンスになっています．別にこのままでもよいのですが，後々のことを考えると，NumPyの配列の方が都合が良いです．
便利なことに，`y_train.values`とすることで，列の名前や行の番号が消え，実際に持っている値だけをNumPyの配列として取り出すことができます．
`y_train`は`Series`クラスのインスタンスになっていますが，これは`DataFrame`クラスでも同じです．
つまり，`d_train.values`とすることで，同じく行番号や列の名前を消して，中の値だけをNumPyの配列として取り出すことができます．

In [8]:
y_train = y_train.values
print(y_train)
print(f"y_trainのクラスは{type(y_train)}")

[1.67 1.53 1.48 ... 1.5  1.41 1.21]
y_trainのクラスは<class 'numpy.ndarray'>


### ここまでのまとめ
- `pandas`の`read_csv`でcsvファイルを読み込むことができる．読み込んだファイルは`DataFrame`クラスのインスタンスとして返される．
- `DataFrame`クラスのインスタンスは`print`で中身を簡単にわかりやすく見ることができる．
- `DataFrame`では，[列の名前]や[列のリスト]とすることで，特定の列を取り出すことができる．
- `DataFrame`では，[i:j]とすることで，特定の範囲の行を取り出すことができる．
- `.values`で`DataFrame`や`DataFrame`から取り出した列の持つ値を，NumPyの配列`np.array`として取り出すことができる．

## 特徴ベクトルを作る

さて，いよいよ特徴ベクトルを作っていきましょう．入力の情報として，以下が与えられています：
 - Date：日付
 - 4046, 4225, 4770：小，大，特大として売れた数
 - Small Bags, Large Bags, XLarge Bags：小，大，特大の袋として売れた数
 - type：オーガニックか否か
 - region：地域

この中で，"Date"，"type"，"region"が文字列（string)の情報です．
しかし，機械学習のアルゴリズムの多くは各データが「数値のベクトル」として表されていると仮定しており，このような**文字列の情報はそのまま扱えません**．
今回動かしてみる線形回帰も，数値の情報しか扱えません．
機械学習や統計では，（基本的には）**一つ一つのデータは数ベクトル**として表現され，**データの集まりは数の行列**として表現されている，と考えます（一つ一つの行が一つのデータに対応し，一つ一つの列が何らかの情報（特徴）に対応）．同様に，**訓練データの目標値（ここではAveragePrice）の集まりは数のベクトル**として表現されていると考えます．
sklearnを使って学習する際も，訓練データの入力を表す行列と，訓練データの目標値を表すベクトルを渡す必要があります．

ここまででも結構な分量になっている気がするので，今回は，"4046"，"4225"，"4770"，"Small Bags"，"Large Bags"，"XLarge Bags"の**元から数値的な情報である6つだけ**を用いることにします．
"Date"，"type"，"region"の文字列で表される情報は次回用いることにします（もちろん，どのように用いるかのアイデアがすでにあり，そのためのプログラムが書けそうであるならば，ぜひ行ってみてください）．


ここまで皆さんは**ただ読んで動かしてきただけで退屈**だったかと思います．**そこでQuizです**．

### Quiz 1

以下のセルを完成させて，`d_train`と`d_test`から"4046"，"4225"，"4770"，"Small Bags"，"Large Bags"，"XLarge Bags"の6つの列を取り出して作った，`X_train_num`と`X_test_num`を作成しなさい．ここで，`X_train_num`と`X_test_num`はどちらもNumPyの配列`np.ndarray`（`np.array`）であるとします．
なお，`columns_num`は負担を削減するためこちらで用意したリストです．使わなくても良いですが，使ったほうが楽かと思います．

**Quizの解答はノートブックの最下部にあります．**

In [10]:
columns_num = ["4046", "4225", "4770", "Small Bags", "Large Bags", "XLarge Bags"]
X_train_num = 
X_test_num = 

できたでしょうか？中身を出力してみます．上の`d_train`や`d_test`と見比べてみてください．
`type`や`shape`の`print`では，
 - 訓練データ：<class 'numpy.ndarray'> (12774, 6)
 - テストデータ：<class 'numpy.ndarray'> (5475, 6)
 
と表示されていればよいです（`.shape`は配列の形を表しています）．

In [11]:
print("X_train_numを表示")
print(type(X_train_num), X_train_num.shape)
print(X_train_num)
print("\nX_test_numを表示")
print(type(X_test_num), X_test_num.shape)
print(X_test_num)

X_train_numを表示
<class 'numpy.ndarray'> (12774, 6)
[[1.12134000e+03 5.18426000e+03 0.00000000e+00 8.88210000e+02
  2.71794000e+03 0.00000000e+00]
 [1.58231547e+06 1.09543356e+06 4.70724600e+04 8.63081250e+05
  5.91149150e+05 2.17492000e+03]
 [1.41410000e+02 7.94480000e+02 0.00000000e+00 5.32050000e+03
  1.45269000e+03 0.00000000e+00]
 ...
 [6.13650000e+02 1.85211000e+03 0.00000000e+00 4.08363000e+03
  2.42858000e+03 0.00000000e+00]
 [5.53480000e+02 5.70000000e+00 0.00000000e+00 4.51472000e+03
  1.13330000e+02 0.00000000e+00]
 [1.56506220e+05 1.60239430e+05 1.73007400e+04 5.87562800e+04
  0.00000000e+00 0.00000000e+00]]

X_test_numを表示
<class 'numpy.ndarray'> (5475, 6)
[[1.9823160e+04 5.8366540e+04 2.0104000e+02 2.6571688e+05 3.0752060e+04
  0.0000000e+00]
 [2.1571200e+03 1.1754369e+05 3.2032100e+03 2.6280320e+04 3.1985380e+04
  0.0000000e+00]
 [8.2729500e+04 3.3601780e+04 1.7903440e+04 2.1493260e+04 6.9886800e+03
  1.2603700e+03]
 ...
 [5.6366000e+03 1.5479570e+04 0.0000000e+00 2.5904000

なお，この配列はそれぞれ列の数が6つ，つまり各データが6つの数値的な情報によって表現されている，ということになります．
このようなデータを表す情報のことを**特徴(feature)**といい，特徴のベクトルのことをそのまま**特徴ベクトル**，特徴の数を**（データの，あるいは特徴の）次元**と言います（つまり，今は6次元の特徴ベクトルとしてデータを表現している，といえる）．

## 線形回帰を動かす
ここまでおつかれさまでした．**ついに機械学習アルゴリズムを動かす**ときが来ました．
sklearnの教師あり学習のモデル・アルゴリズムの実装について，以下に列挙されています：

https://scikit-learn.org/stable/supervised_learning.html#supervised-learning

凄まじい量ですね．色々ありますが，最も単純な手法の一つである**線形回帰（重回帰）=LinearRegression**を動かしてみます．

一応，線形回帰について簡単に説明しておきます．まず，**線形モデル（Linear Model）**は以下のようなモデルです：

$$
 y(\mathbf{x}; \mathbf{w}) = \mathbf{w}^{\top} \mathbf{x} = \sum_{j=1}^{D}w_jx_j,
$$


ここで，$\mathbf{x} \in \mathbb{R}^{D}$は特徴ベクトル，$D$はその次元（今回の例だと$D=6$），$\mathbf{w} \in \mathbb{R}^{D}$は線形モデルの学習されるパラメータです（$x_1$や$x_2$というのが一つの特徴を表しています）．
「**線形モデルの学習**」は「**訓練データから$\mathbf{w}$をイイカンジに定める**」ことを意味します．
線形モデルを回帰に用いるとき，あるいは**平均二乗誤差**$\sum_{n=1}^N (y(\mathbf{x}_n; \mathbf{w})-t_n)^2 / N$を最小化することで学習を行うとき（特にデータサイエンスの講義で習った方法で（初回の「問題」）行うとき），線形モデルは特に**線形回帰**と呼ばれます．
データサイエンスの初回で導入された**多項式回帰**も結局のところ線形回帰とみなすことができます（多項式特徴ベクトルに対する線形回帰）．

**線形モデル**の学習アルゴリズムは一つではなく様々あります．
線形回帰では二乗誤差を最小化しましたが，別の関数を最小化しても良いです．
また，同じ関数を最小化する場合でも，最小化の仕方（アルゴリズム）も一つとは限りません．
また，「何らかの制約を課せられた線形モデル」というのもあったりします．
「二乗誤差を用いた線形モデル」が「線形回帰」と呼ばれるように，学習アルゴリズムが変わるとまた別の名前で呼ばれるようになったりします．
[先程のsklearnの教師あり学習のモデル・アルゴリズムの実装一覧](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)を見ると，**Linear Models**に属している手法が**合計で17個**もありますが，これらは基本的には全てモデル（関数の形）としては同じだけれども，学習アルゴリズム等が違うため分けられています．


線形回帰の学習方法についてはデータサイエンスの授業で習い，実験で実際に定める（学習する）プログラムを書くことになりますが，今回の演習ではあくまでデータ分析全体の流れを理解することなので，学習するプログラムを皆さんに書いてもらうことはありません（もちろん，勉強のために書くことは素晴らしいことだと思いますが）．偉大なるsklearnが勝手にやってくれるのです．

ちょっと話が長くなってしまいました．本題に入ります．`LinearRegression`を使えば線形回帰を動かすことができます．
sklearnを用いる基本的な手順は，
1. モデルのインスタンスを作成
2. 作成したモデルオブジェクトを**`fit`メソッド**を用いて学習．`fit`メソッドには訓練データの入力と目標値（つまり，行列とベクトル）を渡す．
3. 学習したモデルを用いて**`predict`メソッド**で予測．`predict`メソッドにはデータの入力（つまり行列）を渡す．

となっています．スライドでは，プロセスの中に「学習アルゴリズムを選ぶ」というような手順が入っていたと思います．
sklearnでは学習アルゴリズムが違う場合は別のクラスとして提供されていることが多いです（先程の線形モデルの例のように）．
また，一つのクラスの中で学習アルゴリズムを選べる場合も，インスタンスを作成する時に学習アルゴリズムを指定します．
したがって，sklearnでは「モデルのインスタンスを作成」した時点で学習アルゴリズムを選んだことになります．


では実際に上の手順通りに学習してみます．
Pythonにおいて，`ClassA`という名前のクラスのインスタンスは`ClassA()`で作ることができるのでした（day1_how_to_python.ipynb参照）．
したがって，今回のケースでは以下のようになります．

In [12]:
# 手順1：LinearRegressionのインスタンスの作成
lr = LinearRegression()
# 手順2：上で作ったオブジェクトの学習
lr.fit(X_train_num, y_train)
# 手順3：テストデータに対する予測
y_pred_test = lr.predict(X_test_num)

**これで学習とテストデータに対する予測まで終わりました!**たった3行です．すごいですね．予測結果をプリントしてみましょう．

In [13]:
print(y_pred_test)

[1.42770035 1.42778589 1.41099591 ... 1.42480144 1.52284404 1.42431654]


## 予測結果の提出

予測ができたので，この結果を提出することを考えます．予測結果をファイルに出力しなければなりません．ファイル出力ときくと面倒そうだと思うかもしれませんが，NumPyを用いれば以下のように簡単にできます．
`np.savetxt`の`X`という引数で保存したい配列を指定し，`fname`という引数でファイル名を指定します．
今回は，`y_pred_test`を`y_pred_lr.txt`という名前で保存することにします．

In [14]:
np.savetxt(X=y_pred_test, fname='y_pred_lr.txt')

y_pred_lr.txtというファイルが生成されました（Google Colabの場合は，画面左側のタブが「ファイル」のであるとき，「ファイル」という文字の下に3つアイコンがありますが，その真ん中の「更新」アイコンを押すと出てくるはずです）．これを提出してみる（Google Colabの場合はまず生成されたy_pred_lr.txtをダウンロードしてください）と，全てが1.0のy_pred_example.txtより良い結果になると考えられます（ちゃんと学習ができていれば）．

## 線形回帰の修正

線形モデルは以下の式

$$
 y(\mathbf{x}; \mathbf{w}) = \mathbf{w}^{\top} \mathbf{x} = \sum_{j=1}^{D}w_jx_j
$$

で表される，と述べました．あきらかに，どのような$\mathbf{w}$を推定（学習）しても，$\mathbf{x}=\mathbf{0}$のとき予測結果は$0$となってしまいます．
一次元の場合で考えるとわかりやすいです：このモデルは**原点を通る直線**しか表現することができません．
そこで，以下のように新たに**バイアス項（bias term）**あるいは**切片（intercept）**と呼ばれるパラメータ$b \in \mathbb{R}$を導入することがあります：

$$
 y(\mathbf{x}; \mathbf{w}, b) = \mathbf{w}^{\top} \mathbf{x} + b= \sum_{j=1}^{D}w_jx_j + b.
$$

$b$も$\mathbf{w}$と同様に学習するパラメータです．

$b$を導入したほうが良いのか否かはタスクやデータに依存しており，それ自体は学習するものではなく，**機械学習のユーザが決める**ものです．
機械学習のモデルや学習アルゴリズムの中には，このように**学習するのではなくユーザが定める要素**を持つものがあります（今回は「用いるか否か」ですが，連続値や離散値を定めないといけないこともある）．
このような要素（パラメータ）を**ハイパーパラメータ**と言います（もっとも，今回の例では「用いるか否か」であるためあまり**パラメータ感がない**上に，さらに$b=0$と推定されれば「用いない」に対応するため，あまりハイパーパラメータとは感じない（もしかすると言わない）かもしれませんが）．
わかりやすいハイパーパラメータの例としては，データサイエンスの初回の授業で出てきた，**正則化項の強さ$\lambda$**があげられます．

さて，この話を聴くと**$b$を導入するぞ！**と思うのではないでしょうか．
しかし，**実はsklearnの`LinearRegression`ではデフォルトで導入**されています．
そこで，今度は**$b$を導入しない**ことを考えてみます．
sklearnにおいて，モデルのハイパーパラメータはインスタンスを作る際に指定することができます．
ではここでQuizです．

### Quiz 2
[sklearnのLinearRegressionのドキュメント](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)を読んで，「**バイアス項（切片）を使わない線形回帰**」を動かし，学習しなさい．また，テストデータに関する予測結果を`print`し，バイアス項を使った場合（つまり，上の`y_pred_lr`の`print`結果）と結果が変わっていることを確認しなさい．さらに，その予測結果を`y_pred_lr_without_bias.txt`として保存しなさい（提出はしてもしなくてもよい）．

[ 0.10116005  0.03665987 -0.00890221 ...  0.00281757  0.71213428
  0.00324945]


バイアス項を導入した場合（最初に動かした結果）と比べると，予測の値が全体的にかなり小さくなっていますね．
訓練データのAveragePriceと比較すると，この結果は一見すると良さそうには思えません．しかし，**良いか悪いかをパっと見ではなく定量的に評価する必要が本来はあります**．
提出するとスコアが返ってくるので定量的な評価をすることができますが，一日の提出回数が5回に限られているため，一見すると悪そうなものに5回のうちの1回を使ってしまうのはMOTTAINAI気もします．
**提出をする前の定量的な評価**についても次回学んでいきましょう．

今日の資料はここで終了です．おつかれさまでした．
上で述べていますが，sklearnには他にも様々なモデル・アルゴリズムが実装されているので，時間があればぜひ色々試してみてください（授業資料は最低限の手引きくらいの気持ちで）．LinearRegressionだけでもあと一つくらいはいじるパラメータがあると思います．
やる気のある方はぜひ金曜日以外も積極的に取り組んでみてください．


## まとめその2
 - 機械学習のアルゴリズムは（基本的には）数値の情報しか使えない．非数値的な情報はどうにかして数的な情報に変換する必要がある．
 - sklearnでは以下の手順で予測モデルを作成，学習しテストデータの予測を行う：(1)インスタンスを作成し，(2)`fit`メソッドでモデルを学習，(3)`predict`メソッドでテストデータの目標値を予測．
 - `np.savetxt`で予測結果をファイルに書き込むことができる．
 - 多くの予測モデルにはユーザが決めるパラメータがあり，それらをハイパーパラメータという．ハイパーパラメータの値によって結果は大きく変わることもあるため，試行錯誤の余地がある．
 - 「文字列の特徴をどうするか」「提出をする前の定量的な評価」については次回を行う ．

### Quiz 3 (すこし発展的）
情報理工学演習IVの担当教員とTA/TFは，バイアス項付きの線形回帰にひどい恨みがあるようで，**バイアス項付きの設定の`LinearRegression`を使うことを禁止してしまった（注意：してませんが，仮定の話です）**．
しかし，あなたには複雑な事情があり，どうしても，なにがなんでも，どんなことをしてでも**バイアス項付きの線形回帰を使いたい**．
そこで，バイアス項を使わない設定の`LinearRegression`（つまり，Quiz 2のときの設定の`LinearRegression`のインスタンス）で，**工夫を凝らして**，バイアス項付きの線形モデル（つまり，最初に動かした線形モデル`lr=LinearRegression()`）と同じ予測を行えるようにしなさい．

[1.42770035 1.42778589 1.41099591 ... 1.42480144 1.52284404 1.42431654]


## Answers 

### Quiz 1
`DataFrame`に対して，ブラケット（角括弧）[]を使うことで特定の列を取り出せるのでした．また，`.values`で`DataFrame`の値のみを持つ`np.array`を作成できるのでした．

In [17]:
columns_num = ["4046", "4225", "4770", "Small Bags", "Large Bags", "XLarge Bags"]
X_train_num = d_train[columns_num].values
X_test_num = d_test[columns_num].values

### Quiz 2

`LinearRegression`のドキュメントを見ると，`fit_intercept`というパラメータがあり，以下のように説明がされています："Whether to calculate the intercept for this model."
そのため，`fit_intercept=False`を，インスタンスを作る時に指定してあげればよいです．ファイルの作成には`np.savetxt`を使うのでした．

In [18]:
lr = LinearRegression(fit_intercept=False)
lr.fit(X_train_num, y_train)
y_pred_test_without_bias = lr.predict(X_test_num)
print(y_pred_test_without_bias)
np.savetxt(X=y_pred_test_without_bias, fname='y_pred_lr_without_bias.txt')

[ 0.10116005  0.03665987 -0.00890221 ...  0.00281757  0.71213428
  0.00324945]


### Quiz 3
バイアス項付きの線形回帰は，
$$
 y(\mathbf{x}; \mathbf{w}, b) = \mathbf{w}^{\top} \mathbf{x} + b= \sum_{j=1}^{D}w_jx_j + b =  \sum_{j=1}^{D}w_jx_j + b\cdot 1
$$
と書けます（$b=b\times 1$とした）．この$1$を**特徴の値**とみなすと，このバイアス項付きの線形回帰は次のように**バイアス項のない線形回帰**として書けます：
$$
 y(\mathbf{x}; \mathbf{w}, b) = y((x_1, \ldots, x_D, 1)^\top; (w_1, \ldots, w_D, b)^\top).
$$
つまり，「入力が$(x_1, \ldots, x_{D}, 1)^\top$の$D+1$次元ベクトルである，バイアス項なしの線形モデルのパラメータ$\mathbf{w} \in \mathbb{R}^{D+1}$の$D+1$番目の要素」が，「入力が$\mathbf{x}$であるバイアス項付きの線形回帰のバイアス項」に対応します．
したがって，`X_train_num`と`X_test_num`の列の末尾に（正確には，共通の位置ならどこでも良い）にすべてが**$1$の列ベクトル**を追加すれば良いです．

`np.ndarray`の横方向の連結は`np.hstack`で行うことができます．すべてが`1`のベクトルや行列は`np.ones`で作成することができます．
[hstackの使い方](https://numpy.org/doc/stable/reference/generated/numpy.hstack.html)，[onesの使い方](https://numpy.org/doc/1.18/reference/generated/numpy.ones.html)はそれぞれのドキュメントや前回の資料を参照のこと．
結局，以下のようになります．

In [19]:
X_train_num_hstack1 = np.hstack([X_train_num, np.ones((n_train, 1))]) # 列の数が6から7になり，最後の列がすべて1
X_test_num_hstack1 = np.hstack([X_test_num, np.ones((n_test, 1))]) # 列の数が6から7になり，最後の列がすべて1
lr = LinearRegression(fit_intercept=False) # fit_intercept=Falseとすることを忘れないように！
lr.fit(X_train_num_hstack1, y_train) # 新しく作った行列を使う
y_pred_test = lr.predict(X_test_num_hstack1) # 新しく作った行列を使う
print(y_pred_test)

[1.42770035 1.42778589 1.41099591 ... 1.42480144 1.52284404 1.42431654]


なお，sklearnには`add_dummy_feature`という関数があり，それを用いるとより簡単にできます．以下のようになります．

In [20]:
from sklearn.preprocessing import add_dummy_feature
X_train_num_hstack1 = add_dummy_feature(X_train_num, 1.0) 
X_test_num_hstack1 = add_dummy_feature(X_test_num, 1.0)
lr = LinearRegression(fit_intercept=False) # fit_intercept=Falseとすることを忘れないように！
lr.fit(X_train_num_hstack1, y_train) # 新しく作った行列を使う
y_pred_test = lr.predict(X_test_num_hstack1) # 新しく作った行列を使う
print(y_pred_test)

[1.42770035 1.42778589 1.41099591 ... 1.42480144 1.52284404 1.42431654]
